# Environment Setup

In [1]:
import yaml
import gymnasium as gym
import numpy as np 
from types import SimpleNamespace as SN
from pathlib import Path
import copy
import utils.common_utils as cu
from algos.ddpg_agent import DDPGAgent
from algos.ppo_agent import PPOAgent
from utils.recorder import RecordVideo
from algos.ddpg_extension import DDPGExtension
from algos.ppo_extension import PPOExtension
import torch


In [2]:

# Function to test a trained policy
def test(agent, env_name, algo_name):
    # Load model
    agent.load_model()
    print("Testing...")
    total_test_reward, total_test_len = 0, 0
    returns = []
    
    cur_dir=Path().cwd()
    cfg_path= cur_dir/'cfg'
    # read configuration parameters:
    cfg={'cfg_path': cfg_path, 'algo_name': algo_name}
    env_cfg=yaml.safe_load(open(cfg_path /'envs'/f'{env_name}_env.yaml', 'r'))
    
    # prepare folders to store results
    work_dir = cur_dir/'results'/env_cfg["env_name"]/algo_name
    video_test_dir=work_dir/"video"/"test"
    
    for ep in range(agent.cfg.test_episodes):
        frames = []
        seed = np.random.randint(low=1, high=1000)
        observation, _ = agent.env.reset(seed=seed)
        test_reward, test_len, done = 0, 0, False
        
        while not done and test_len < agent.cfg.max_episode_steps:
            action, _ = agent.get_action(observation, evaluation=True)
            observation, reward, done, truncated, info = agent.env.step(action.flatten())
            fs = agent.env.render()
            frames = frames+fs
            test_reward += reward
            test_len += 1
        total_test_reward += test_reward
        total_test_len += test_len
        returns.append(test_reward)
        
        if ep%100==0:
            cu.save_rgb_arrays_to_gif(frames, video_test_dir/('_seed_'+str(agent.seed)+'_ep_'+str(ep)+'.gif'))

    print(f"Average test reward over {len(returns)} episodes: {float(total_test_reward/agent.cfg.test_episodes)},+- {np.std(np.array(returns))}; \
        Average episode length: {total_test_len/agent.cfg.test_episodes}")


In [3]:
# Setup: read the configurations and generate the environment.
def setup(algo=None, env='easy', cfg_args={}, render=True, train_episodes=None):
    # set the paths
    cur_dir=Path().cwd()
    cfg_path= cur_dir/'cfg'
    
    # read configuration parameters:
    cfg={'cfg_path': cfg_path, 'algo_name': algo}
    env_cfg=yaml.safe_load(open(cfg_path /'envs'/f'{env}_env.yaml', 'r'))
    algo_cfg=yaml.safe_load(open(cfg_path /'algo'/f'{algo}.yaml', 'r'))
    cfg.update(env_cfg)
    cfg.update(algo_cfg)
    cfg.update(cfg_args)
    
    # forcely change train_episodes
    if train_episodes is None:
        True
    else:
        cfg["train_episodes"] = train_episodes
    
    # prepare folders to store results
    work_dir = cur_dir/'results'/cfg["env_name"]/str(algo)
    model_dir=work_dir/"model"
    logging_dir=work_dir/"logging"
    video_train_dir=work_dir/"video"/"train"
    video_test_dir=work_dir/"video"/"test"
    for dir in [work_dir, model_dir, logging_dir, video_train_dir, video_test_dir]:
        cu.make_dir(dir)
        
    cfg.update({'work_dir':work_dir, "model_dir":model_dir, "logging_dir": logging_dir, "video_train_dir": video_train_dir, "video_test_dir": video_test_dir})
    cfg = SN(**cfg)
    
    # set seed
    if cfg.seed == None:
        seed = np.random.randint(low=1, high=1000)
    else:
        seed = cfg.seed
    
    ## Create environment
    env=cu.create_env(cfg_path /'envs'/f'{env}_env.yaml')

   
    if cfg.save_video:
        # During testing, save every episode
        if cfg.testing:
            ep_trigger = 1
            video_path = cfg.video_test_dir
        # During training, save every 50th episode
        else:
            ep_trigger = 1000   # Save video every 50 episodes
            video_path = cfg.video_train_dir
        
        if render:
            env = RecordVideo(
                env, video_path,
                episode_trigger=lambda x: x % ep_trigger == 0,
                name_prefix=cfg.exp_name)


    eval_env=copy.deepcopy(env)
    env.reset(seed=seed) # we only set the seed here. During training, we don't have to set the seed when performing reset().
    eval_env.reset(seed=seed+1000)
    eval_env=None # For simplicity, we don't evaluate the performance during training.
        
    # Get dimensionalities of actions and observations
    action_space_dim = cu.get_space_dim(env.action_space)
    observation_space_dim = cu.get_space_dim(env.observation_space)
    
    config={
        "args": cfg,
        "env":env,
        "eval_env":eval_env,
        "action_space_dim": action_space_dim,
        "observation_space_dim": observation_space_dim,
        "seed":seed
    }
    return config


# DDPG

## Middle

### Training

In [4]:
# Choose either PPO or DDPG
implemented_algo ='ddpg' #'ppo' or 'ddpg'

# Loop over the three difficulty levels
# for environment in ['easy', 'middle', 'difficult']:
environment = 'middle'
training_seeds = []

# Train the algorithm with a specific random seed.
# In total, we train the algorithm with three random seeds [0, 1, 2].
for i in range(3):
    config=setup(algo=implemented_algo, env=environment)

    config["seed"] = i
    training_seeds.append(i)
    
    # torch.manual_seed(config["seed"])
    # np.random.seed(config["seed"])
    if config["args"].algo_name == 'ppo':
        agent=PPOAgent(config)
    elif config["args"].algo_name == 'ddpg':
        agent=DDPGAgent(config)
    else:
        raise Exception('Please use ppo or ddpg!')

    # Train the agent using selected algorithm    
    agent.train()


/Users/lgk1910/Library/CloudStorage/OneDrive-AaltoUniversity/Learning2/ReinforcementLearning/Project/GitHub/Reinforcement-Learning-Project/utils/recorder.py:250: UserWarning: WARN: Overwriting existing videos at /Users/lgk1910/Library/CloudStorage/OneDrive-AaltoUniversity/Learning2/ReinforcementLearning/Project/GitHub/Reinforcement-Learning-Project/results/SandingEnvMiddle/ddpg/video/train folder (try specifying a different `video_folder` for the `RecordVideo` wrapper if this is not desired)
  logger.warn(
/Users/lgk1910/anaconda3/envs/rl/lib/python3.10/site-packages/gymnasium/core.py:311: UserWarning: WARN: env.is_vector_env to get variables from other wrappers is deprecated and will be removed in v1.0, to get this variable you can do `env.unwrapped.is_vector_env` for environment variables or `env.get_wrapper_attr('is_vector_env')` that will search the reminding wrappers.
  logger.warn(
/Users/lgk1910/anaconda3/envs/rl/lib/python3.10/site-packages/gymnasium/utils/passive_env_checker.p

Moviepy - Building video /Users/lgk1910/Library/CloudStorage/OneDrive-AaltoUniversity/Learning2/ReinforcementLearning/Project/GitHub/Reinforcement-Learning-Project/results/SandingEnvMiddle/ddpg/video/train/project-episode-0.mp4.
Moviepy - Writing video /Users/lgk1910/Library/CloudStorage/OneDrive-AaltoUniversity/Learning2/ReinforcementLearning/Project/GitHub/Reinforcement-Learning-Project/results/SandingEnvMiddle/ddpg/video/train/project-episode-0.mp4



Moviepy - Done !
Moviepy - video ready /Users/lgk1910/Library/CloudStorage/OneDrive-AaltoUniversity/Learning2/ReinforcementLearning/Project/GitHub/Reinforcement-Learning-Project/results/SandingEnvMiddle/ddpg/video/train/project-episode-0.mp4


/Users/lgk1910/anaconda3/envs/rl/lib/python3.10/site-packages/gymnasium/utils/passive_env_checker.py:135: UserWarning: WARN: The obs returned by the `step()` method was expecting numpy array dtype to be float32, actual type: float64
  logger.warn(
/Users/lgk1910/anaconda3/envs/rl/lib/python3.10/site-packages/gymnasium/utils/passive_env_checker.py:159: UserWarning: WARN: The obs returned by the `step()` method is not within the observation space.
  logger.warn(f"{pre} is not within the observation space.")


Moviepy - Building video /Users/lgk1910/Library/CloudStorage/OneDrive-AaltoUniversity/Learning2/ReinforcementLearning/Project/GitHub/Reinforcement-Learning-Project/results/SandingEnvMiddle/ddpg/video/train/project-episode-0.mp4.
Moviepy - Writing video /Users/lgk1910/Library/CloudStorage/OneDrive-AaltoUniversity/Learning2/ReinforcementLearning/Project/GitHub/Reinforcement-Learning-Project/results/SandingEnvMiddle/ddpg/video/train/project-episode-0.mp4



Moviepy - Done !
Moviepy - video ready /Users/lgk1910/Library/CloudStorage/OneDrive-AaltoUniversity/Learning2/ReinforcementLearning/Project/GitHub/Reinforcement-Learning-Project/results/SandingEnvMiddle/ddpg/video/train/project-episode-0.mp4
Episode 0 Step 20 finished. Average episode return: 0.0
Episode 100 Step 2020 finished. Average episode return: -0.01
Episode 200 Step 4020 finished. Average episode return: 0.07
Episode 300 Step 6020 finished. Average episode return: -0.21
Episode 400 Step 8020 finished. Average episode return: -0.1
Episode 500 Step 10020 finished. Average episode return: 0.29
Episode 600 Step 12020 finished. Average episode return: 0.33
Episode 700 Step 14020 finished. Average episode return: 0.56
Episode 800 Step 16020 finished. Average episode return: 0.44
Episode 900 Step 18020 finished. Average episode return: 0.22
Moviepy - Building video /Users/lgk1910/Library/CloudStorage/OneDrive-AaltoUniversity/Learning2/ReinforcementLearning/Project/GitHub/Reinforcement

Moviepy - Done !
Moviepy - video ready /Users/lgk1910/Library/CloudStorage/OneDrive-AaltoUniversity/Learning2/ReinforcementLearning/Project/GitHub/Reinforcement-Learning-Project/results/SandingEnvMiddle/ddpg/video/train/project-episode-1000.mp4
Episode 1000 Step 20020 finished. Average episode return: 0.54
Episode 1100 Step 22020 finished. Average episode return: 0.58


KeyboardInterrupt: 

In [ ]:
## Code block for training and testing an agent using the implemented algorithm
## in the three different Tasks with different difficulty levels
import warnings
warnings.filterwarnings('ignore')

# NOTE: Uncomment the algorithm you implemented
implemented_algo ='ddpg' #'ddpg' or 'ppo'

environment = 'middle'
training_seeds = []

# for each algorithm, we will test the agent trained with specific random seed
for i in range(3):
    config=setup(algo=implemented_algo, env=environment, render=False)

    config["seed"] = i
    training_seeds.append(i)

    if config["args"].algo_name == 'ppo':
        agent=PPOAgent(config)
    elif config["args"].algo_name == 'ddpg':
        agent=DDPGAgent(config)
    else:
        raise Exception('Please use ppo or ddpg!')
    
    print('\n\n\nnow start testing for environment',environment,' agent:',implemented_algo,' seed:',i)
    # Test the agent in the selected environment
    test(agent, environment, implemented_algo)




now start testing for environment middle  agent: ddpg  seed: 0
model loaded: /Users/lgk1910/Library/CloudStorage/OneDrive-AaltoUniversity/Learning2/ReinforcementLearning/Project/GitHub/Reinforcement-Learning-Project/results/SandingEnvMiddle/ddpg/model/model_parameters_0.pt
Testing...
Saved GIF to /Users/lgk1910/Library/CloudStorage/OneDrive-AaltoUniversity/Learning2/ReinforcementLearning/Project/GitHub/Reinforcement-Learning-Project/results/SandingEnvMiddle/ddpg/video/test/_seed_0_ep_0.gif
Average test reward over 10 episodes: 1.3,+- 0.6403124237432849;         Average episode length: 20.0



now start testing for environment middle  agent: ddpg  seed: 1
model loaded: /Users/lgk1910/Library/CloudStorage/OneDrive-AaltoUniversity/Learning2/ReinforcementLearning/Project/GitHub/Reinforcement-Learning-Project/results/SandingEnvMiddle/ddpg/model/model_parameters_1.pt
Testing...
Saved GIF to /Users/lgk1910/Library/CloudStorage/OneDrive-AaltoUniversity/Learning2/ReinforcementLearning/Projec

# DDPG Extension

## Middle

### TD3

#### Training

In [ ]:
# Implement your improved algorithm either in algo/ddpg_extension.py or algo/ppo_extension.py
from algos.ddpg_extension import DDPGExtension
# from algos.ddpg_extension_QR import DDPGExtension
from algos.ppo_extension import PPOExtension

implemented_algo = 'ddpg_extension'# choose 'ppo_extension' or 'ddpg_extension'
environment = 'middle'

training_seeds = []
for i in range(3):
    config=setup(algo=implemented_algo, env=environment)

    config["seed"] = i
    training_seeds.append(i)

    if config["args"].algo_name == 'ppo_extension':
        agent=PPOExtension(config)
    elif config["args"].algo_name == 'ddpg_extension':
        agent=DDPGExtension(config)
    else:
        raise Exception('Please use ppo or ddpg!')

    # Train the agent using selected algorithm    
    agent.train()

#### Testing

In [ ]:
from algos.ddpg_extension import DDPGExtension
from algos.ppo_extension import PPOExtension

implemented_algo = 'ddpg_extension'# choose 'ppo_extension' or 'ddpg_extension'
environment = 'middle'
training_seeds = []
for i in range(3):
    config=setup(algo=implemented_algo, env=environment, render=False)

    config["seed"] = i
    training_seeds.append(i)


    if config["args"].algo_name == 'ppo_extension':
        agent=PPOExtension(config)
    elif config["args"].algo_name == 'ddpg_extension':
        agent=DDPGExtension(config)
    else:
        raise Exception('Please use ppo or ddpg!')

    # Test the agent in the selected environment
    test(agent, environment, implemented_algo)

### TD3 + RS

#### TRAINING

In [4]:
# Implement your improved algorithm either in algo/ddpg_extension.py or algo/ppo_extension.py
from algos.ddpg_extension_TD3_RS import DDPGExtension
from algos.ppo_extension import PPOExtension
import torch

implemented_algo = 'ddpg_extension_TD3_RS'# choose 'ppo_extension' or 'ddpg_extension'
environment = 'middle'

training_seeds = []
for i in range(3):
    config=setup(algo=implemented_algo, env=environment)

    config["seed"] = i
    training_seeds.append(i)

    if config["args"].algo_name == 'ppo_extension':
        agent=PPOExtension(config)
    elif config["args"].algo_name == 'ddpg_extension_TD3_RS':
        agent=DDPGExtension(config)
    else:
        raise Exception('Please use ppo or ddpg!')

    # Train the agent using selected algorithm    
    agent.train()


/Users/lgk1910/Library/CloudStorage/OneDrive-AaltoUniversity/Learning2/ReinforcementLearning/Project/GitHub/Reinforcement-Learning-Project/utils/recorder.py:250: UserWarning: WARN: Overwriting existing videos at /Users/lgk1910/Library/CloudStorage/OneDrive-AaltoUniversity/Learning2/ReinforcementLearning/Project/GitHub/Reinforcement-Learning-Project/results/SandingEnvMiddle/ddpg_extension_TD3_RS/video/train folder (try specifying a different `video_folder` for the `RecordVideo` wrapper if this is not desired)
  logger.warn(
/Users/lgk1910/anaconda3/envs/rl/lib/python3.10/site-packages/gymnasium/core.py:311: UserWarning: WARN: env.is_vector_env to get variables from other wrappers is deprecated and will be removed in v1.0, to get this variable you can do `env.unwrapped.is_vector_env` for environment variables or `env.get_wrapper_attr('is_vector_env')` that will search the reminding wrappers.
  logger.warn(
/Users/lgk1910/anaconda3/envs/rl/lib/python3.10/site-packages/gymnasium/utils/pass

Moviepy - Building video /Users/lgk1910/Library/CloudStorage/OneDrive-AaltoUniversity/Learning2/ReinforcementLearning/Project/GitHub/Reinforcement-Learning-Project/results/SandingEnvMiddle/ddpg_extension_TD3_RS/video/train/project-episode-0.mp4.
Moviepy - Writing video /Users/lgk1910/Library/CloudStorage/OneDrive-AaltoUniversity/Learning2/ReinforcementLearning/Project/GitHub/Reinforcement-Learning-Project/results/SandingEnvMiddle/ddpg_extension_TD3_RS/video/train/project-episode-0.mp4



Moviepy - Done !
Moviepy - video ready /Users/lgk1910/Library/CloudStorage/OneDrive-AaltoUniversity/Learning2/ReinforcementLearning/Project/GitHub/Reinforcement-Learning-Project/results/SandingEnvMiddle/ddpg_extension_TD3_RS/video/train/project-episode-0.mp4
Using DDPGExtension with TD3 + RS


/Users/lgk1910/anaconda3/envs/rl/lib/python3.10/site-packages/gymnasium/core.py:311: UserWarning: WARN: env.n_sanding to get variables from other wrappers is deprecated and will be removed in v1.0, to get this variable you can do `env.unwrapped.n_sanding` for environment variables or `env.get_wrapper_attr('n_sanding')` that will search the reminding wrappers.
  logger.warn(
/Users/lgk1910/anaconda3/envs/rl/lib/python3.10/site-packages/gymnasium/core.py:311: UserWarning: WARN: env.n_no_sanding to get variables from other wrappers is deprecated and will be removed in v1.0, to get this variable you can do `env.unwrapped.n_no_sanding` for environment variables or `env.get_wrapper_attr('n_no_sanding')` that will search the reminding wrappers.
  logger.warn(
/Users/lgk1910/anaconda3/envs/rl/lib/python3.10/site-packages/gymnasium/core.py:311: UserWarning: WARN: env.MAX_SIZE to get variables from other wrappers is deprecated and will be removed in v1.0, to get this variable you can do `env.unw

Moviepy - Building video /Users/lgk1910/Library/CloudStorage/OneDrive-AaltoUniversity/Learning2/ReinforcementLearning/Project/GitHub/Reinforcement-Learning-Project/results/SandingEnvMiddle/ddpg_extension_TD3_RS/video/train/project-episode-0.mp4.
Moviepy - Writing video /Users/lgk1910/Library/CloudStorage/OneDrive-AaltoUniversity/Learning2/ReinforcementLearning/Project/GitHub/Reinforcement-Learning-Project/results/SandingEnvMiddle/ddpg_extension_TD3_RS/video/train/project-episode-0.mp4



...

Moviepy - Done !
Moviepy - video ready /Users/lgk1910/Library/CloudStorage/OneDrive-AaltoUniversity/Learning2/ReinforcementLearning/Project/GitHub/Reinforcement-Learning-Project/results/SandingEnvMiddle/ddpg_extension_TD3_RS/video/train/project-episode-6000.mp4
Episode 6000 Step 120020 finished. Average episode return: 0.79
Episode 6100 Step 122020 finished. Average episode return: 0.97
Episode 6200 Step 124020 finished. Average episode return: 0.86
Episode 6300 Step 126020 finished. Average episode return: 0.89
Episode 6400 Step 128020 finished. Average episode return: 0.84
Episode 6500 Step 130020 finished. Average episode return: 0.93
Episode 6600 Step 132020 finished. Average episode return: 0.94
Episode 6700 Step 134020 finished. Average episode return: 0.83
Episode 6800 Step 136020 finished. Average episode return: 0.78
Episode 6900 Step 138020 finished. Average episode return: 0.82
Moviepy - Building video /Users/lgk1910/Library/CloudStorage/OneDrive-AaltoUniversity/Learning2/Re

Moviepy - Done !
Moviepy - video ready /Users/lgk1910/Library/CloudStorage/OneDrive-AaltoUniversity/Learning2/ReinforcementLearning/Project/GitHub/Reinforcement-Learning-Project/results/SandingEnvMiddle/ddpg_extension_TD3_RS/video/train/project-episode-7000.mp4
Episode 7000 Step 140020 finished. Average episode return: 0.96
Episode 7100 Step 142020 finished. Average episode return: 0.98
Episode 7200 Step 144020 finished. Average episode return: 0.92
Episode 7300 Step 146020 finished. Average episode return: 1.0
Episode 7400 Step 148020 finished. Average episode return: 0.87
Episode 7500 Step 150020 finished. Average episode return: 0.87
Episode 7600 Step 152020 finished. Average episode return: 0.78
Episode 7700 Step 154020 finished. Average episode return: 0.81
Episode 7800 Step 156020 finished. Average episode return: 0.76
Episode 7900 Step 158020 finished. Average episode return: 0.92
Moviepy - Building video /Users/lgk1910/Library/CloudStorage/OneDrive-AaltoUniversity/Learning2/Rei

Moviepy - Done !
Moviepy - video ready /Users/lgk1910/Library/CloudStorage/OneDrive-AaltoUniversity/Learning2/ReinforcementLearning/Project/GitHub/Reinforcement-Learning-Project/results/SandingEnvMiddle/ddpg_extension_TD3_RS/video/train/project-episode-8000.mp4
Episode 8000 Step 160020 finished. Average episode return: 0.98
Episode 8100 Step 162020 finished. Average episode return: 0.83
Episode 8200 Step 164020 finished. Average episode return: 0.82
Episode 8300 Step 166020 finished. Average episode return: 0.83
Episode 8400 Step 168020 finished. Average episode return: 0.75
Episode 8500 Step 170020 finished. Average episode return: 0.83
Episode 8600 Step 172020 finished. Average episode return: 0.92
Episode 8700 Step 174020 finished. Average episode return: 0.79
Episode 8800 Step 176020 finished. Average episode return: 0.86
Episode 8900 Step 178020 finished. Average episode return: 0.91
Moviepy - Building video /Users/lgk1910/Library/CloudStorage/OneDrive-AaltoUniversity/Learning2/Re

Moviepy - Done !
Moviepy - video ready /Users/lgk1910/Library/CloudStorage/OneDrive-AaltoUniversity/Learning2/ReinforcementLearning/Project/GitHub/Reinforcement-Learning-Project/results/SandingEnvMiddle/ddpg_extension_TD3_RS/video/train/project-episode-9000.mp4
Episode 9000 Step 180020 finished. Average episode return: 0.91
Episode 9100 Step 182020 finished. Average episode return: 0.97
Episode 9200 Step 184020 finished. Average episode return: 0.92
Episode 9300 Step 186020 finished. Average episode return: 0.85
Episode 9400 Step 188020 finished. Average episode return: 0.94
Episode 9500 Step 190020 finished. Average episode return: 0.83
Episode 9600 Step 192020 finished. Average episode return: 0.94
Episode 9700 Step 194020 finished. Average episode return: 0.84
Episode 9800 Step 196020 finished. Average episode return: 0.88
Episode 9900 Step 198020 finished. Average episode return: 0.94
Moviepy - Building video /Users/lgk1910/Library/CloudStorage/OneDrive-AaltoUniversity/Learning2/Re

Moviepy - Done !
Moviepy - video ready /Users/lgk1910/Library/CloudStorage/OneDrive-AaltoUniversity/Learning2/ReinforcementLearning/Project/GitHub/Reinforcement-Learning-Project/results/SandingEnvMiddle/ddpg_extension_TD3_RS/video/train/project-episode-10000.mp4
Episode 10000 Step 200020 finished. Average episode return: 0.94
Saved model to /Users/lgk1910/Library/CloudStorage/OneDrive-AaltoUniversity/Learning2/ReinforcementLearning/Project/GitHub/Reinforcement-Learning-Project/results/SandingEnvMiddle/ddpg_extension_TD3_RS/model/model_parameters_2.pt ...
logger and seed 2
/Users/lgk1910/Library/CloudStorage/OneDrive-AaltoUniversity/Learning2/ReinforcementLearning/Project/GitHub/Reinforcement-Learning-Project/results/SandingEnvMiddle/ddpg_extension_TD3_RS/logging/logs_2.csv
------ Training Finished ------
Total traning time is 10.971739736116433mins


In [5]:
from algos.ddpg_extension_TD3_RS import DDPGExtension
from algos.ppo_extension import PPOExtension

implemented_algo = 'ddpg_extension_TD3_RS'# choose 'ppo_extension' or 'ddpg_extension'
environment = 'middle'
training_seeds = []
for i in range(3):
    config=setup(algo=implemented_algo, env=environment, render=False)

    config["seed"] = i
    training_seeds.append(i)


    if config["args"].algo_name == 'ppo_extension':
        agent=PPOExtension(config)
    elif config["args"].algo_name == 'ddpg_extension_TD3_RS':
        agent=DDPGExtension(config)
    else:
        raise Exception('Please use ppo or ddpg!')

    # Test the agent in the selected environment
    test(agent, environment, implemented_algo)

Using DDPGExtension with TD3 + RS
model loaded: /Users/lgk1910/Library/CloudStorage/OneDrive-AaltoUniversity/Learning2/ReinforcementLearning/Project/GitHub/Reinforcement-Learning-Project/results/SandingEnvMiddle/ddpg_extension_TD3_RS/model/model_parameters_0.pt
Testing...
Saved GIF to /Users/lgk1910/Library/CloudStorage/OneDrive-AaltoUniversity/Learning2/ReinforcementLearning/Project/GitHub/Reinforcement-Learning-Project/results/SandingEnvMiddle/ddpg_extension_TD3_RS/video/test/_seed_0_ep_0.gif
Average test reward over 10 episodes: 0.8,+- 0.6;         Average episode length: 20.0
Using DDPGExtension with TD3 + RS
model loaded: /Users/lgk1910/Library/CloudStorage/OneDrive-AaltoUniversity/Learning2/ReinforcementLearning/Project/GitHub/Reinforcement-Learning-Project/results/SandingEnvMiddle/ddpg_extension_TD3_RS/model/model_parameters_1.pt
Testing...
Saved GIF to /Users/lgk1910/Library/CloudStorage/OneDrive-AaltoUniversity/Learning2/ReinforcementLearning/Project/GitHub/Reinforcement-Learn

## Difficult

### TD3 + RS

#### Training

In [4]:
# Implement your improved algorithm either in algo/ddpg_extension.py or algo/ppo_extension.py
from algos.ddpg_extension_TD3_RS import DDPGExtension
from algos.ppo_extension import PPOExtension

implemented_algo = 'ddpg_extension_TD3_RS'# choose 'ppo_extension' or 'ddpg_extension'
environment = 'difficult' # choose 'easy', 'middle' or 'difficult'

training_seeds = []
for i in range(3):
    config=setup(algo=implemented_algo, env=environment)

    config["seed"] = i
    training_seeds.append(i)

    if config["args"].algo_name == 'ppo_extension':
        agent=PPOExtension(config)
    elif config["args"].algo_name == 'ddpg_extension_TD3_RS':
        agent=DDPGExtension(config)
    else:
        raise Exception('Please use ppo or ddpg!')

    # Train the agent using selected algorithm    
    agent.train()


/Users/lgk1910/Library/CloudStorage/OneDrive-AaltoUniversity/Learning2/ReinforcementLearning/Project/GitHub/Reinforcement-Learning-Project/utils/recorder.py:250: UserWarning: WARN: Overwriting existing videos at /Users/lgk1910/Library/CloudStorage/OneDrive-AaltoUniversity/Learning2/ReinforcementLearning/Project/GitHub/Reinforcement-Learning-Project/results/SandingEnvDifficult/ddpg_extension_TD3_RS/video/train folder (try specifying a different `video_folder` for the `RecordVideo` wrapper if this is not desired)
  logger.warn(
/Users/lgk1910/anaconda3/envs/rl/lib/python3.10/site-packages/gymnasium/core.py:311: UserWarning: WARN: env.is_vector_env to get variables from other wrappers is deprecated and will be removed in v1.0, to get this variable you can do `env.unwrapped.is_vector_env` for environment variables or `env.get_wrapper_attr('is_vector_env')` that will search the reminding wrappers.
  logger.warn(
/Users/lgk1910/anaconda3/envs/rl/lib/python3.10/site-packages/gymnasium/utils/p

Moviepy - Building video /Users/lgk1910/Library/CloudStorage/OneDrive-AaltoUniversity/Learning2/ReinforcementLearning/Project/GitHub/Reinforcement-Learning-Project/results/SandingEnvDifficult/ddpg_extension_TD3_RS/video/train/project-episode-0.mp4.
Moviepy - Writing video /Users/lgk1910/Library/CloudStorage/OneDrive-AaltoUniversity/Learning2/ReinforcementLearning/Project/GitHub/Reinforcement-Learning-Project/results/SandingEnvDifficult/ddpg_extension_TD3_RS/video/train/project-episode-0.mp4



Moviepy - Done !
Moviepy - video ready /Users/lgk1910/Library/CloudStorage/OneDrive-AaltoUniversity/Learning2/ReinforcementLearning/Project/GitHub/Reinforcement-Learning-Project/results/SandingEnvDifficult/ddpg_extension_TD3_RS/video/train/project-episode-0.mp4
Using DDPGExtension with TD3 + RS


/Users/lgk1910/anaconda3/envs/rl/lib/python3.10/site-packages/gymnasium/core.py:311: UserWarning: WARN: env.n_sanding to get variables from other wrappers is deprecated and will be removed in v1.0, to get this variable you can do `env.unwrapped.n_sanding` for environment variables or `env.get_wrapper_attr('n_sanding')` that will search the reminding wrappers.
  logger.warn(
/Users/lgk1910/anaconda3/envs/rl/lib/python3.10/site-packages/gymnasium/core.py:311: UserWarning: WARN: env.n_no_sanding to get variables from other wrappers is deprecated and will be removed in v1.0, to get this variable you can do `env.unwrapped.n_no_sanding` for environment variables or `env.get_wrapper_attr('n_no_sanding')` that will search the reminding wrappers.
  logger.warn(
/Users/lgk1910/anaconda3/envs/rl/lib/python3.10/site-packages/gymnasium/core.py:311: UserWarning: WARN: env.MAX_SIZE to get variables from other wrappers is deprecated and will be removed in v1.0, to get this variable you can do `env.unw

Moviepy - Building video /Users/lgk1910/Library/CloudStorage/OneDrive-AaltoUniversity/Learning2/ReinforcementLearning/Project/GitHub/Reinforcement-Learning-Project/results/SandingEnvDifficult/ddpg_extension_TD3_RS/video/train/project-episode-0.mp4.
Moviepy - Writing video /Users/lgk1910/Library/CloudStorage/OneDrive-AaltoUniversity/Learning2/ReinforcementLearning/Project/GitHub/Reinforcement-Learning-Project/results/SandingEnvDifficult/ddpg_extension_TD3_RS/video/train/project-episode-0.mp4



...

Moviepy - Done !
Moviepy - video ready /Users/lgk1910/Library/CloudStorage/OneDrive-AaltoUniversity/Learning2/ReinforcementLearning/Project/GitHub/Reinforcement-Learning-Project/results/SandingEnvDifficult/ddpg_extension_TD3_RS/video/train/project-episode-6000.mp4
Episode 6000 Step 120020 finished. Average episode return: 0.79
Episode 6100 Step 122020 finished. Average episode return: 1.17
Episode 6200 Step 124020 finished. Average episode return: 1.02
Episode 6300 Step 126020 finished. Average episode return: 1.23
Episode 6400 Step 128020 finished. Average episode return: 1.04
Episode 6500 Step 130020 finished. Average episode return: 1.09
Episode 6600 Step 132020 finished. Average episode return: 0.92
Episode 6700 Step 134020 finished. Average episode return: 1.08
Episode 6800 Step 136020 finished. Average episode return: 1.01
Episode 6900 Step 138020 finished. Average episode return: 1.09
Moviepy - Building video /Users/lgk1910/Library/CloudStorage/OneDrive-AaltoUniversity/Learning2

Moviepy - Done !
Moviepy - video ready /Users/lgk1910/Library/CloudStorage/OneDrive-AaltoUniversity/Learning2/ReinforcementLearning/Project/GitHub/Reinforcement-Learning-Project/results/SandingEnvDifficult/ddpg_extension_TD3_RS/video/train/project-episode-7000.mp4
Episode 7000 Step 140020 finished. Average episode return: 0.96
Episode 7100 Step 142020 finished. Average episode return: 1.33
Episode 7200 Step 144020 finished. Average episode return: 1.02
Episode 7300 Step 146020 finished. Average episode return: 1.01
Episode 7400 Step 148020 finished. Average episode return: 1.17
Episode 7500 Step 150020 finished. Average episode return: 0.97
Episode 7600 Step 152020 finished. Average episode return: 0.94
Episode 7700 Step 154020 finished. Average episode return: 1.02
Episode 7800 Step 156020 finished. Average episode return: 1.09
Episode 7900 Step 158020 finished. Average episode return: 0.92
Moviepy - Building video /Users/lgk1910/Library/CloudStorage/OneDrive-AaltoUniversity/Learning2

Moviepy - Done !
Moviepy - video ready /Users/lgk1910/Library/CloudStorage/OneDrive-AaltoUniversity/Learning2/ReinforcementLearning/Project/GitHub/Reinforcement-Learning-Project/results/SandingEnvDifficult/ddpg_extension_TD3_RS/video/train/project-episode-8000.mp4
Episode 8000 Step 160020 finished. Average episode return: 0.91
Episode 8100 Step 162020 finished. Average episode return: 1.16
Episode 8200 Step 164020 finished. Average episode return: 1.16
Episode 8300 Step 166020 finished. Average episode return: 0.91
Episode 8400 Step 168020 finished. Average episode return: 0.99
Episode 8500 Step 170020 finished. Average episode return: 0.95
Episode 8600 Step 172020 finished. Average episode return: 0.95
Episode 8700 Step 174020 finished. Average episode return: 0.84
Episode 8800 Step 176020 finished. Average episode return: 1.04
Episode 8900 Step 178020 finished. Average episode return: 1.16
Moviepy - Building video /Users/lgk1910/Library/CloudStorage/OneDrive-AaltoUniversity/Learning2

Moviepy - Done !
Moviepy - video ready /Users/lgk1910/Library/CloudStorage/OneDrive-AaltoUniversity/Learning2/ReinforcementLearning/Project/GitHub/Reinforcement-Learning-Project/results/SandingEnvDifficult/ddpg_extension_TD3_RS/video/train/project-episode-9000.mp4
Episode 9000 Step 180020 finished. Average episode return: 0.9
Episode 9100 Step 182020 finished. Average episode return: 0.92
Episode 9200 Step 184020 finished. Average episode return: 1.1
Episode 9300 Step 186020 finished. Average episode return: 0.95
Episode 9400 Step 188020 finished. Average episode return: 1.01
Episode 9500 Step 190020 finished. Average episode return: 1.02
Episode 9600 Step 192020 finished. Average episode return: 0.94
Episode 9700 Step 194020 finished. Average episode return: 1.09
Episode 9800 Step 196020 finished. Average episode return: 1.06
Episode 9900 Step 198020 finished. Average episode return: 0.92
Moviepy - Building video /Users/lgk1910/Library/CloudStorage/OneDrive-AaltoUniversity/Learning2/R

Moviepy - Done !
Moviepy - video ready /Users/lgk1910/Library/CloudStorage/OneDrive-AaltoUniversity/Learning2/ReinforcementLearning/Project/GitHub/Reinforcement-Learning-Project/results/SandingEnvDifficult/ddpg_extension_TD3_RS/video/train/project-episode-10000.mp4
Episode 10000 Step 200020 finished. Average episode return: 1.03
Saved model to /Users/lgk1910/Library/CloudStorage/OneDrive-AaltoUniversity/Learning2/ReinforcementLearning/Project/GitHub/Reinforcement-Learning-Project/results/SandingEnvDifficult/ddpg_extension_TD3_RS/model/model_parameters_2.pt ...
logger and seed 2
/Users/lgk1910/Library/CloudStorage/OneDrive-AaltoUniversity/Learning2/ReinforcementLearning/Project/GitHub/Reinforcement-Learning-Project/results/SandingEnvDifficult/ddpg_extension_TD3_RS/logging/logs_2.csv
------ Training Finished ------
Total traning time is 6.594687868750043mins


#### Testing

In [5]:
training_seeds = []
environment = 'difficult'
for i in range(3):
    config=setup(algo=implemented_algo, env=environment, render=False)

    config["seed"] = i
    training_seeds.append(i)

    if config["args"].algo_name == 'ppo_extension':
        agent=PPOExtension(config)
    elif config["args"].algo_name == 'ddpg_extension_TD3_RS':
        agent=DDPGExtension(config)
    else:
        raise Exception('Please use ppo or ddpg!')

    # Test the agent in the selected environment
    test(agent, environment, implemented_algo)

Using DDPGExtension with TD3 + RS
model loaded: /Users/lgk1910/Library/CloudStorage/OneDrive-AaltoUniversity/Learning2/ReinforcementLearning/Project/GitHub/Reinforcement-Learning-Project/results/SandingEnvDifficult/ddpg_extension_TD3_RS/model/model_parameters_0.pt
Testing...
Saved GIF to /Users/lgk1910/Library/CloudStorage/OneDrive-AaltoUniversity/Learning2/ReinforcementLearning/Project/GitHub/Reinforcement-Learning-Project/results/SandingEnvDifficult/ddpg_extension_TD3_RS/video/test/_seed_0_ep_0.gif
Average test reward over 10 episodes: 0.9,+- 1.044030650891055;         Average episode length: 20.0
Using DDPGExtension with TD3 + RS
model loaded: /Users/lgk1910/Library/CloudStorage/OneDrive-AaltoUniversity/Learning2/ReinforcementLearning/Project/GitHub/Reinforcement-Learning-Project/results/SandingEnvDifficult/ddpg_extension_TD3_RS/model/model_parameters_1.pt
Testing...
Saved GIF to /Users/lgk1910/Library/CloudStorage/OneDrive-AaltoUniversity/Learning2/ReinforcementLearning/Project/Git